# Interactivity

This notebook walks through a simple, yet powerful example of how our abstract interactivity system can be used to link graphics and their features together.

In [1]:
import fastplotlib as fpl
from sidecar import Sidecar
import numpy as np
from ipywidgets import VBox

Load in some neural data.

In [2]:
# reconstructed movie of CaImAn Sue demo movie after running motion correction and CNMF algorithms
movie = np.load('./fpl-scipy2023-data/neural_data/rcm.npy')

# temporal traces of identified neurons
temporal = np.load('./fpl-scipy2023-data/neural_data/temporal.npy')

# contours of identified neurons
contours = np.load("./fpl-scipy2023-data/neural_data/contours.npy", allow_pickle=True)

Will need a euclidean helper function to indentify which contours has been clicked on. We hope to soon include this, and other common callback functions, in our interactivity system :D

In [3]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

### Plot creation

Create an `ImageWidget`to display the reconstructed movie and identified neurons. Also create a `Plot` instance to display the associated temporal data for each identified nueron.

In [4]:
# for the image data and contours
cnmf_iw = fpl.ImageWidget(
    movie, 
    vmin_vmax_sliders=True, 
    grid_plot_kwargs={"size": (600, 400)},
    cmap="gnuplot2",
)

# temporal plot
plot_temporal = fpl.Plot(size=(600,100))
plot_temporal.add_line(temporal[0], colors="magenta")

# stack the plots and show them in sidecar
sc = Sidecar(title="interactivity")

with sc:
    display(VBox([cnmf_iw.show(), plot_temporal.show(maintain_aspect=False)]))

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


RFBOutputContext()

In [5]:
# add contours
contours_graphic = cnmf_iw.gridplot[0,0].add_line_collection(
    contours, 
    colors="red", 
    thickness=2, 
    name="contours"
)

In [6]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

### Interacitvity of `Graphics` using `link()` and callback functions

In [7]:
# link image to contours
image_graphic = cnmf_iw.gridplot[0,0]["image_widget_managed"]

image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="cyan", 
    callback=euclidean
)

# a vertical line that is syncronized to the image widget "t" (timepoint) slider
def update_timepoint(ev):
    ix = ev.pick_info["selected_index"]
    cnmf_iw.sliders["t"].value = ix


# callback function to display correct temporal trace
def generate_temporal(ev):
    # clear the plot 
    plot_temporal.clear()
    
    # get data of selected ix
    data = temporal[ev.pick_info["collection-index"]]
    
    # add trace to plot 
    temporal_graphic = plot_temporal.add_line(data, colors="magenta")
    
    # add selector synced to "t" slider
    selector = temporal_graphic.add_linear_selector()
    selector.selection.add_event_handler(update_timepoint)
    
    plot_temporal.auto_scale()

# add event handler so that temporal trace is generated when contour is clicked on
contours_graphic[:].colors.add_event_handler(generate_temporal)

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

In [8]:
plot_temporal.close()
cnmf_iw.gridplot.close()
sc.close()